<a href="https://colab.research.google.com/github/gersonvida12-hash/Desconstrutor-de-Prompts---Usa-top-tecnologias/blob/main/Generative_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @phase_0: SETUP
!pip install -q crewai python-dotenv langchain-groq
!mkdir -p tools agents tasks
!touch main.py agents.py tasks.py tools/c.py

# @phase_1: WRITE ALL FILES
%%writefile tools/c.py
from crewai_tools import BaseTool as BT
class CTool(BT):
  name="ClarifyTool";description="Ask user questions."
  def _run(self,q:list[str])->str:
    r=input("\n".join([f"Q{i}:{s}" for i,s in enumerate(q,1)]));return f"User:{r}"

%%writefile agents.py
from crewai import Agent as A;from tools.c import CTool
def mk_a():
  d=A(role='Diagnostician',goal='P0/1:Analyze,RCA',tools=[CTool()],allow_delegation=False)
  r=A(role='RiskAnalyst',goal='P1:Pre-mortem',allow_delegation=False)
  s=A(role='Architect',goal='P2:Synthesize solution',allow_delegation=False)
  return d,r,s

%%writefile tasks.py
from crewai import Task as T
def mk_t(ags,p):
  d,r,s=ags
  t1=T(description=f"Analyze:{p}",expected_output="JSON:vector,rca,context",agent=d)
  t2=T(description="Risk analysis",expected_output="MD_TABLE:Risk Matrix",agent=r,context=[t1])
  t3=T(description="Build HYPERION_PACKAGE",expected_output="MD_BLOCK:Final package",agent=s,context=[t2])
  return[t1,t2,t3]

%%writefile main.py
import os;from crewai import Crew,Process as P;from agents import mk_a;from tasks import mk_t;from langchain_groq import ChatGroq;from google.colab import userdata
try:os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
except:os.environ['GROQ_API_KEY']=input("GROQ_API_KEY?:")
llm=ChatGroq(model_name="llama3-70b-8192")
def run(p):
  ags=mk_a();[setattr(a,'llm',llm) for a in ags];tasks=mk_t(ags,p)
  crew=Crew(agents=list(ags),tasks=tasks,process=P.sequential)
  return crew.kickoff()
if __name__=="__main__":
  p=input("PROBLEM?:");res=run(p);print(f"\n---PACKAGE---\n{res}")

UsageError: Line magic function `%%writefile` not found.


In [ ]:
!python main.py

Building a generative AI model involves several steps, from gathering data to deploying your model in the real world.

**Step 1:** Gathering Data
The very first step is to collect the right data for your project. Here’s how you can do it:


*   **Data Selection**: Decide what you want your model to create, whether it’s stories, poems, or even responses in a chatbot.
*   **Data Collection**: Find lots of examples of what you want to generate. For text, you can gather books, articles, or conversations from the internet.

**Step 2:** Preprocessing Your Data
Before feeding your data to the AI model, you need to prepare it:


*   **Cleaning**: Remove any messy or irrelevant parts from your data.
*   **Tokenization**: Split your text into smaller chunks, like words or sentences.
*   **Normalization**: Ensure everything is consistent; for text, it means converting everything to lowercase.

**Step 3**: Choosing a Generative Model Architecture
Like 1. Recurrent Neural Networks (RNNs) 2. Generative Adversarial Networks (GANs)

**Step 4**: Training Your Generative Model
Training is where your AI model learns to be creative:


*   **Batch Size**: Experiment with different batch sizes to find what works best for your model.
*   **Training Time**: Be patient; training can take a while, especially with lots of data.
*   **Regularization**: Use techniques like dropout to prevent your model from getting too obsessed with the training data.

**Step 5**: Evaluating and Fine-Tuning
After training, it’s time to assess your model’s performance and make it even better:


*  **Evaluation Metrics**: Determine how to measure the quality of what your model generates. For text, you might use metrics like “how human-like is it?”
*   **Fine-Tuning**: Based on your evaluation, tweak your model. Adjust settings, get more data, or change the architecture.

**Step 6**: Deploying Your Generative Model
Once your model is ready, it’s time to put it to work:


*   **API Integration**: If you want others to use your model, create an API so they can interact with it easily.
*   **Monitoring**: Keep an eye on how your model performs in the real world and update it when needed.

In [ ]:
!pip install tensorflow

**Step 1**: Gathering Data
Let’s assume you have a list of sentences as your dataset:

In [ ]:
data = ["This is the first sentence.", "Here's the second sentence.", "Finally, the third one."]

**Step 2**: Preprocessing Your Data

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)
total_words = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(data)

input_sequences = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        n_gram_sequence = sequence[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

X = input_sequences[:, :-1]
y = input_sequences[:, -1]
print("done with preprocessing data")

**Step 3**: Choosing a Generative Model Architecture
Let’s set up an RNN for text generation:

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 64, input_length=max_sequence_length-1),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

**Step 4**: Training Your Generative Model

In [ ]:
model.fit(X, y, epochs=100, verbose=1)

**Step 5**: Generating Text

In [ ]:
import numpy as np

seed_text = "This is"
next_words = 10

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    # predicted = model.predict_classes(token_list, verbose=0)
    # predicted = (model.predict(token_list) > 0.5).astype("int32")
    predicted = np.argmax(model.predict(token_list),axis=1)

    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)